In [1]:
import os, sys
import numpy as np
import dask
import xarray as xr
import xgcm

from matplotlib import pyplot as plt
%matplotlib inline

from mitequinox.utils import *

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/cmocean/tools.py:76: MatplotlibDeprecationWarning: The is_string_like function was deprecated in version 2.1.
  if not mpl.cbook.is_string_like(rgbin[0]):


In [2]:
from dask_jobqueue import PBSCluster
local_dir = os.getenv('TMPDIR')
cluster = PBSCluster(local_directory=local_dir)
#print(cluster.job_script())
w = cluster.start_workers(10)

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/distributed/utils.py:128: RuntimeWarning: Couldn't detect a suitable IP address for reaching '8.8.8.8', defaulting to '127.0.0.1': [Errno 101] Network is unreachable
  % (host, default, e), RuntimeWarning)


In [3]:
# get dask handles and check dask server status
from dask.distributed import Client
client = Client(cluster)

In [6]:
client

Client Scheduler: tcp://10.148.0.195:48596 Dashboard: http://10.148.0.195:8787/status,Cluster Workers: 2 Cores: 8 Memory: 100.00 GB


--- 

# For each hour

In [5]:
V = ['SST']

out_dir = '/home1/scratch/xyu/mit_zarr/'  
Nt = 1

for v in V:

    #
    data_dir = root_data_dir+v+'/'
    iters, time = get_iters_time(v, data_dir, delta_t=25.)
    it = np.arange(5).astype(int)*Nt # tmp

    assert it[-1]+Nt<time.size
    #
    p = 'C'
    vdrop = ['XC','YC','Depth','rA','CS','SN']
    if v is 'SSU':
        p = 'W'
        vdrop = ['dxC','dyG','rAw']         
    elif v is 'SSV':
        p = 'S'
        vdrop = ['dxG','dyC','rAs']        
        #
    ds = get_compressed_data(v, data_dir, grid_dir, iters=iters, time=time, client=client, point=p)
        #
        
    for i, t in enumerate(it):
        #
        file_out = out_dir+'%s_t%04d.zarr'%(v,i)
        dv = ds[v].isel(time=slice(t,t+Nt)) 
        dv = dv.drop(vdrop).to_dataset()
        dv.to_zarr(file_out, mode='w')     

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:735: UserWarning: Couldn't find available_diagnostics.log in . Using default version.
  "in %s. Using default version." % data_dir)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/utils.py:336: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:249: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:


_____
# For each face

In [7]:
V = ['SST']
out_dir = '/home1/scratch/xyu/mit_zarr/'  
face_all = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] # some key faces

for face in face_all :
    
    for v in V:

        data_dir = root_data_dir+v+'/'
        iters, time = get_iters_time(v, data_dir, delta_t=25.)
    #
        p = 'C'
        vdrop = ['XC','YC','CS','SN','rA','Depth']
        if v is 'SSU':
            p = 'W'
        elif v is 'SSV':
            p = 'S'
    #
        ds = get_compressed_data(v, data_dir, grid_dir, iters=iters, time=time, client=client, point=p)
        dv = ds[v].isel(time=slice(1,720),face=face)
        dv = dv.chunk({'time': 1,'i':4320,'j':4320})
        dv = dv.drop(vdrop).to_dataset()
        file_out = out_dir+'/%s_f%02d.zarr'%(v,face)
        dv.to_zarr(file_out, mode='w')                    
        dsize = getsize(file_out)
        print('face=%d - allready processed'%(face))

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:735: UserWarning: Couldn't find available_diagnostics.log in . Using default version.
  "in %s. Using default version." % data_dir)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/utils.py:336: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:249: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:


<xarray.DataArray 'SST' (time: 719, j: 4320, i: 4320)>
dask.array<shape=(719, 4320, 4320), dtype=>f4, chunksize=(1, 4320, 4320)>
Coordinates:
  * i        (i) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * j        (j) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
    face     int64 0
    XC       (j, i) >f4 dask.array<shape=(4320, 4320), chunksize=(4320, 4320)>
    YC       (j, i) >f4 dask.array<shape=(4320, 4320), chunksize=(4320, 4320)>
    CS       (j, i) >f4 dask.array<shape=(4320, 4320), chunksize=(4320, 4320)>
    SN       (j, i) >f4 dask.array<shape=(4320, 4320), chunksize=(4320, 4320)>
    rA       (j, i) >f4 dask.array<shape=(4320, 4320), chunksize=(4320, 4320)>
    Depth    (j, i) >f4 dask.array<shape=(4320, 4320), chunksize=(4320, 4320)>
  * time     (time) float64 5.706e+06 5.71e+06 5.713e+06 5.717e+06 5.72e+06 ...
    dtime    (time) datetime64[ns] dask.array<shape=(719,), chunksize=(1,)>
face=0 - allready processed


In [9]:
cluster.scheduler.close()

<Future finished result=None>